# Предобработка данных о продажах игр


### Цели и задачи проекта

Цель - изучить исходные данные, проверить их корректность и провести предобработку, получив необходимый срез данных.    
Задачи:
- Загрузить и оценить данные
- Проверить данные на наличие ошибок и предобработать их (метки столбцов, типы данных, наличие пропусков, явные и неявные дубликаты)
- Провести фильтрацию, оставив только необходимый срез данных
- Категоризировать данные по оценкам пользователей и экспертов

### Описание данных

В проекте будут использованы данные датасета `new_games.csv`, который содержит информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.


### Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод
---

## 1. Загрузка данных и знакомство с ними


In [1]:
# Импортируем библиотеку pandas
import pandas as pd

In [2]:
# Выгружаем данные из датасета new_games.csv в датафрейм df
df = pd.read_csv('/datasets/new_games.csv')

In [3]:
# Выводим первые строки датафрейма на экран
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Выводим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [15]:
# Выводим информацию о количестве уникальных значений в некоторых столбцах
[x + ' - ' + str(len(df[x].value_counts())) for x in ['Platform', 'Genre', 'Rating']]

['Platform - 31', 'Genre - 24', 'Rating - 8']

Можно сделать вывод, что в столбцах `Platform`, `Genre`, `Rating` содержится ограниченное количество уникальных значений, а значит лучше для них выбрать категориальный тип данных.

Датасет `new_games.csv` содержит 11 столбцов и 16956 строк, в которых представлена информация о продажах различных игр.

Изучим типы данных и их корректность:
- **Числовые значения с плавающей точкой (float64).** Четыре столбца представлены типом `float64`:
    - `NA sales`, `Other sales` содержат продажи в миллионах копий и представлены типом `float64`. Это верное решение, как как величина, измеряемая в миллионах, может содержать дробные значения.
    - `Year of Release` хранит информацию о годе выпуска игры. Для таких данных рекомендуется использовать тип `datetime64`, который облегчит выполнение временных вычислений и упростит анализ временных периодов.
    - `Critic Score` содержит целочисленную оценку критиков от 0 до 100. Выбранный тип данных является корректным, однако для целых значений лучше использовать тип данных `int64`.
- **Строковые данные (object).** Семь столбцов представлены типом `object`:
    - `Name` содержит строковую информацию(названия игр), что логично для текстовых данных. Здесь тип данных `object` подходит.
    - `EU sales`, `JP sales`, `User Score` хранят информацию о количестве продаж игр в соответствующих странах и оценке пользоваталей. Для такх данных лучше использовать тип `float64`, который облегчит процесс вычислений.
    - `Platform`, `Genre`, `Rating`хранят текстовые данные, но лучше рассматривать их как категориальные признаки. В этом случае можно использовать тип `category`, чтобы улучшить производительность и оптимизировать память, так как набор значений ограничен(от 8 до 31) и заведомо известен.

Оценки критиков и оценки пользователей представлены в разных шкалах - от 0 до 100 и от 0 до 10 соответственно, что может быть вызвать трудности при анализе. Стоит привести эти столбцы к единой шкале, например, от 0 до 100 и поменять тип данных на `int64`.
В столбцах `Name`, `Platform`, `Genre`, `Year of Release` содержится незначительное количество пропусков - **менее 2%**.
А столбцы `User Score`, `Critic Score`, `Rating` имеют большое количество пропусков - **более 68%**.
Также для удобства стоит привести все названия столбцов к единому стилю snake case.